In [536]:
import pybind11
import os
import tempfile
from uuid import uuid4
import sys
import setuptools
import importlib
from string import Template
import subprocess
import numpy as np

In [406]:
from IPython.core.magic import register_cell_magic, register_line_cell_magic

In [471]:
@register_line_cell_magic
def versor(line, cell=None):
    if cell is not None:
        function_impl = cell
        function, function_name = compile_(function_impl)
        globals()[function_name] = function
    else:
        function_impl = Template('auto f() {return $body;}').substitute(body=line)
        function, _ = compile_(function_impl)
        return function()

In [515]:
@register_cell_magic
def cpp(line, cell=None):
    print(line)
    print(cell)

In [516]:
%%cpp
kfkfkf

fff
kfkfkf


In [477]:
result = %versor (Vec(1,2,3).null() <= Vec(1,3,8,7,3).null())[0]
print(result)

auto f() {return (Vec(1,2,3).null() <= Vec(1,3,8,7,3).null())[0];}
-13.0


In [457]:
%%versor 
pyversor::c3d::multivector_t f() {
    return pyversor::c3d::vector_t(1.,0909.,3.).null();
}

pyversor::c3d::multivector_t f() {
    return pyversor::c3d::vector_t(1.,0909.,3.).null();
}


In [458]:
f()

Multivector [ 0 1 909 3 1 4.131e+05 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]

In [556]:
def generate_source(module_name, function_name, function_impl):
    return Template("""
#include <pybind11/pybind11.h>
#include <pyversor/pyversor.h>

using Multivector = pyversor::c3d::multivector_t;
using Vec = pyversor::c3d::vector_t;
using Biv = pyversor::c3d::bivector_t;
using Vec3 = pyversor::ega::vector_t;
using Rot = pyversor::ega::rotator_t;

namespace py = pybind11;

$function_impl

PYBIND11_MODULE($module_name, m) {

    m.def("$function_name", &$function_name);

}
""").substitute(module_name=module_name,
                function_name=function_name,
                function_impl=function_impl)

In [529]:
%%versor -f
auto g() -> Multivector {
    auto R = Rot(1,2,3,4);
    auto a = Vec3(1,2,3);
    return R * a * ~R;
}

In [541]:
%%versor
auto h() -> Multivector {
    auto R = Rot(1,2,3,4);
    auto a = Vec3(1,2,3);
    return a.spin(R);
}

In [564]:
%timeit -o h()

563 ns ± 5.42 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


<TimeitResult : 563 ns ± 5.42 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)>

In [545]:
%timeit g()

554 ns ± 2.7 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [546]:
R = np.eye(3)
a = np.random.randn(3,1)

In [547]:
%timeit R.dot(a)

470 ns ± 5.52 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [548]:
g()

Multivector [ 0 30 -60 -90 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]

In [549]:
def c(a):
    print(a)

In [552]:
import pyversor

In [562]:
s = pyversor.c3d.Vector.__dict__
print(s['__mul__']())

NotImplemented


In [555]:
%%versor 
auto mul(const Vec& v, const B& b) {
    return v * b;
}

In [ ]:
a = pyversor.c3d.

In [527]:
cmakelists_template = Template(r"""
cmake_minimum_required(VERSION 3.2)
project($name LANGUAGES CXX)

set(CMAKE_CXX_FLAGS "$${CMAKE_CXX_FLAGS} -std=c++14 -O3 -ftemplate-depth=1200")

find_package(pybind11)

pybind11_add_module($name $name.cpp)

""")

In [373]:
python_includes = subprocess.check_output(['python',
                                           '-m', 'pybind11', '--includes']).decode('utf-8').split()
extension_suffix = subprocess.check_output(['python-config', 
                                            '--extension-suffix']).decode('utf-8').replace('\n','')

In [522]:
def compile_(function_impl):
    
#     print(function_impl)
    
    # Extract function name from implementation
    l = function_impl.splitlines()[0]
    function_name = l[l.find(' ')+1:l.find('(')]
    # Generate uuid
    uuid = str(uuid4().hex[:4])
    # Generate unique module name
    module_name = "pyversor_{}_{}".format(function_name, uuid)
    
#     tempdir = tempfile.mkdtemp()
#     print(tempdir)
    with tempfile.TemporaryDirectory() as tempdir:
        source_cpp = module_name + '.cpp'
        source_path = os.path.join(tempdir, source_cpp)
        with open(source_path, 'w') as f:
            f.write(generate_source(module_name, function_name, function_impl))

        build_command = ['clang++', '-shared', '-std=c++14', '-fPIC', '-ftemplate-depth=1200']
        build_command += ['-O3']
#         build_command += ['-ffast-math']
        build_command += python_includes
        build_command += ['-I/home/lars/devel/pyversor/include']
        build_command += [source_cpp]
        build_command += ['-o', module_name + extension_suffix]

    #     print(' '.join(build_command))

        build_output = subprocess.check_output(build_command, cwd=tempdir)

        # Load compiled module
        spec = importlib.util.spec_from_file_location(
            module_name, os.path.join(tempdir, module_name+extension_suffix))
        m = importlib.util.module_from_spec(spec)
        spec.loader.exec_module(m)
        # register in global namespace
        function = m.__dict__[function_name]
        return function, function_name
